<a href="https://colab.research.google.com/github/ed765super/DATA602-Baltimore-Employee-Data-Classification/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
#https://www.kaggle.com/aashofteh/train-dataset-kaggle-credit-scoring-1
df = pd.read_csv('Baltimore_City_Employee_Salaries.csv', sep=',')
print(df.info())
print(df.describe(include='all'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154187 entries, 0 to 154186
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   lastName       153777 non-null  object 
 1   middleInitial  121195 non-null  object 
 2   firstName      154187 non-null  object 
 3   jobClass       154003 non-null  object 
 4   agencyName     154187 non-null  object 
 5   agencyID       154187 non-null  object 
 6   annualSalary   154187 non-null  float64
 7   grossPay       154187 non-null  float64
 8   hireDate       154185 non-null  object 
 9   fiscalYear     154187 non-null  object 
 10  ObjectId       154187 non-null  int64  
dtypes: float64(2), int64(1), object(8)
memory usage: 12.9+ MB
None
       lastName middleInitial firstName  ...    hireDate fiscalYear       ObjectId
count    153777        121195    154187  ...      154185     154187  154187.000000
unique    11645            61      6890  ...        6930         11   

#Background Information/Buisness Question

#EDA


I will start by checking whether any of my features have null values

In [ ]:
print(df.isna().sum())

In [ ]:
df = df.query('[TARGET VARIABLE].notna()', engine='python')
df

Next I will check the 5 number summary for each of my features

In [ ]:
df.describe().T

Next I will gather a visual summary of my data

In [ ]:
df.hist(figsize=(14,14))
plt.show()

Next I'll check for any multicoliniarity

In [ ]:
import seaborn as sns

sns.heatmap(df.corr())
plt.show()

#Data cleaning and Feature engineering


#Modeling


####Splitting Training and Test sets

In [ ]:
from sklearn.model_selection import train_test_split
curr_Random_State = 50

feature_names = df.columns[:-1]
#The :-1 excludes the last column which is the target column.

#Pre:
#X = a valid index list representing columns
#y = a valid column representing the target cass
#state = a positive numeric integer representing the current "random state"
#Post: returns two sets of training-test splits.
def create_splits(x, y, state):
    return (train_test_split(x, y, test_size=.25, random_state = state)
    )
tX_training, tX_test, ty_training, ty_test = create_splits(df[feature_names], df['Class'], curr_Random_State)
print(f'Training samples: {tX_training.shape[0]}')
print(f'Test samples: {tX_test.shape[0]}')

Now I will fit Fit a logistic regression, decision tree, and SVM using grid search

First I will set up the pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
#First grab all the columns and put em in lists
nums = df.dtypes[df.dtypes != 'object'].index.tolist()
cats = df.dtypes[df.dtypes == 'object'].index.tolist()

#Now remove the columns I dont think I need
nums = [x for x in nums if x not in ['arr_delay']]
cats = [x for x in cats if x not in ['tailnum']]

num_pipeline = Pipeline([('impute_missing', SimpleImputer(strategy='median')),
                           ('standardize_num', StandardScaler())
                        ])

cat_pipeline = Pipeline([('impute_missing_cats', SimpleImputer(strategy='most_frequent')),
                          ('create_dummies_cats', OneHotEncoder(handle_unknown='ignore'))])

processing_pipeline = ColumnTransformer(transformers=[('proc_numeric', num_pipeline, nums),
                                                      ('create_dummies', cat_pipeline, cats)])

print('Pipeline Created')

Logistic Regression with grid search

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import time
start_time = time.time()
p1 = Pipeline([('processing', processing_pipeline),
             ('lr', LogisticRegression(solver='liblinear'))])

#Select various correlation strengths with a magnitude of 10
params = {'lr__C': [.001, 0.01, 0.1, 1, 10, 100]}

#I will score based on accurancy as that the dataset is not heavily skewed
lr_gscv = GridSearchCV(p1, param_grid=params, cv=10, scoring='accuracy', refit=True)

lr_gscv = lr_gscv.fit(tX_train, ty_train)

print(f'Validation score: {lr_gscv.best_score_:.2%}')

lr_pred = lr_gscv.predict(tX_test)

print(f'Test score: {lr_gscv.score(tX_test, ty_test):.2%}')
print ("Runtime: ", time.time() - start_time, " seconds")

Validation score: 68.77%
Test score: 67.68%
Runtime:  13.320276260375977  seconds


Descision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import time
start_time = time.time()

p2 = Pipeline([('processing', processing_pipeline),
             ('dt', DecisionTreeClassifier())])

params = {'dt__max_depth': [1, 5, 10, 15, 25],
         'dt__min_samples_split': [3, 10, 15]}

dt_gscv = GridSearchCV(p2, param_grid=params, cv=10, scoring='accuracy', refit=True)
dt_gscv = dt_gscv.fit(tX_train, ty_train)

print(f'Validation score: {dt_gscv.best_score_:.2%}')

dt_pred = dt_gscv.predict(tX_test)

print(f'Test score: {dt_gscv.score(tX_test, ty_test):.2%}')
print ("Runtime: ", time.time() - start_time, " seconds")

Validation score: 64.59%
Test score: 62.70%
Runtime:  43.282294034957886  seconds


SVM

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import time
start_time = time.time()
svm_m = modeling_pipeline = Pipeline([('processing', processing_pipeline),
                                      ('scaling', StandardScaler(with_mean=False)),
                                       ('model', SVC())])


param_grid = [
  {'model__C': [0.1, 1, 10, 100], 'model__kernel': ['rbf']}
 ]

svm_results = GridSearchCV(estimator=svm_m, param_grid=param_grid, cv=10, scoring='accuracy', refit=True)
svm_results = svm_results.fit(tX_train, ty_train)

print(f'Validation score: {svm_results.best_score_:.2%}')

svm_pred = svm_results.predict(tX_test)

print(f'Test score: {svm_results.score(tX_test, ty_test):.2%}')
print ("Runtime: ", time.time() - start_time, " seconds")

Validation score: 64.92%
Test score: 65.09%
Runtime:  195.7040295600891  seconds


#Results and discussion


#Fit an ensemble using the three above models.

In [ ]:

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.pipeline import Pipeline
import numpy as np

clf1 = LogisticRegression(penalty='l2', C=0.001, solver='lbfgs', random_state=0)
clf2 = DecisionTreeClassifier(max_depth=1, criterion='entropy', random_state=1)
clf3 = svm.SVC(probability=True)

pipe1 = Pipeline([('processing', processing_pipeline),
             ('lr', LogisticRegression(solver='liblinear'))])
pipe2 = Pipeline([('processing', processing_pipeline),
             ('dt', DecisionTreeClassifier())])
pipe3 = Pipeline([('processing', processing_pipeline),
                ('scaling', StandardScaler(with_mean=False)),
                ('model', SVC())])

labs = ['Logistic Regression', 'Decision Tree', 'Support Vector Machine']
clfs = [pipe1, pipe2, pipe3]
clfs = zip(labs, clfs)

for lab, clf in clfs:
    scores = cross_val_score(estimator=clf, X=tX_train, y=ty_train, cv=10, scoring='accuracy')
    print(f'Accuracy {scores.mean():.2f} (+/- {scores.std():.2f}) [{lab}]')

Accuracy 0.68 (+/- 0.02) [Logistic Regression]
Accuracy 0.59 (+/- 0.02) [Decision Tree]
Accuracy 0.63 (+/- 0.01) [Support Vector Machine]


In [ ]:
from sklearn.ensemble import VotingClassifier

ems = [('lr', pipe1),('dt', pipe2),('model', pipe3)]
#Changing voting to hard because SVM doesnt use probabilities
#so using the mode is ok
clf4 = VotingClassifier(estimators= ems, weights=None, voting='hard')

scores = cross_val_score(estimator=clf4, X=tX_train, y=ty_train, cv=10, scoring='accuracy')
print(f'Accuracy {scores.mean():.2f} (+/- {scores.std():.2f}) [Ensemble]')

Accuracy 0.66 (+/- 0.01) [Ensemble]


ADA BOOST

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import time
start_time = time.time()
# multi-step pipelines don't play as nice with AdaBoost
xt = processing_pipeline.fit_transform(tX_train)

params = {'n_estimators': [50, 100, 200, 500, 1000]
         }

#ABC = AdaBoostClassifier(base_estimator=p2)
ABC = AdaBoostClassifier(DecisionTreeClassifier())


ad_gscv = GridSearchCV(ABC, param_grid = params, cv=10, scoring='accuracy')
           
ad_gscv = ad_gscv.fit(xt, ty_train)

print(f'Validation score: {ad_gscv.best_score_:.2%}')

xtt = processing_pipeline.transform(tX_test)

ad_pred = ad_gscv.predict(xtt)

print(f'Test score: {ad_gscv.score(xtt, ty_test):.2%}')
print ("Runtime: ", time.time() - start_time, " seconds")

Validation score: 58.57%
Test score: 58.34%
Runtime:  23.792025566101074  seconds


#Discuss the performance of each model.
* Logistic Regression had the highest accuracy score and took the least amount of time to run
* SVM took the longest despite searching over less correlation strengths. This is most likely due to the immense ammount of copies being created during SVM
* Descision Trees had the lowest accuracy score

#Conclusion/next steps


#References